In [7]:
import weaviate
import os
from dotenv import load_dotenv
import json


# Load the .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")


# Instantiate the client with the auth config
client = weaviate.Client(
    url="https://first-cluster-p4w5zezq.weaviate.network",  # Replace w/ your endpoint
    auth_client_secret=weaviate.AuthApiKey(api_key=WEAVIATE_API_KEY), 
    additional_headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY
    },
)

# Schema


In [10]:
class_obj = {
    "class": "Article",
    "properties": [
        {
            "dataType": ["text"],
            "name": "title",
        },
    ],
    "vectorizer": "text2vec-openai"  # This could be any vectorizer
}

client.schema.create_class(class_obj)

In [13]:
client.schema.get()

{'classes': [{'class': 'Question',
   'description': "This property was generated by Weaviate's auto-schema feature on Mon May 15 09:36:38 2023",
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'properties': [{'dataType': ['text'],
     'description': "This property was generated by Weaviate's auto-schema feature on Mon May 15 09:36:38 2023",
     'indexFilterable': True,
     'indexSearchable': True,
     'name': 'answer',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': "This property was generated by Weaviate's auto-schema feature on Mon May 15 09:36:38 2023",
     'indexFilterable': True,
     'indexSearchable': True,
     'name': 'question',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': "This property was generated by Weaviate's auto-schema feature on Mon May 15 09:36:38 2023",
     'indexFilterable': Tr

: 

# Load and import data

In [11]:
# Load data
import requests
url = 'https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Configure a batch process
with client.batch as batch:
    batch.batch_size=100
    # Batch import all Questions
    for i, d in enumerate(data):
        print(f"importing question: {i+1}")

        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }

        client.batch.add_data_object(properties, "Question")

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


# Query weaviate

In [12]:
nearText = {"concepts": ["biology"]}

result = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text(nearText)
    .with_limit(2)
    .do()
)

print(json.dumps(result, indent=4))

{
    "errors": [
        {
            "locations": [
                {
                    "column": 24,
                    "line": 1
                }
            ],
            "message": "Unknown argument \"nearText\" on field \"Question\" of type \"GetObjectsObj\". Did you mean \"nearVector\" or \"nearObject\"?",
            "path": null
        }
    ]
}
